## Merge sorted streams
Write a generator function that takes two sorted streams (generators), and return a generator that can produce a merged stream in sorted order.

Bonus point: can you make it generic such that it can merge any number of streams?


```python
def merge_sorted_streams(stream1, stream2):
    yield ...

stream1 = range(0, 10, 2)
stream2 = range(1, 10, 2)

for x in merge_sorted_stream(stream1, stream2):
    print(x)

0 
1
2
3
4
5
6
7
8
9
```

In [53]:
import heapq

def merge_sorted_streams(*streams):
    if not streams:
        raise StopIteration

    if len(streams) == 1:
        for val in streams[0]:
            yield val
        return

    # make sure the streams are iterators
    streams = [iter(stream) for stream in streams]
    # flag indicating whether the stream has exhausted
    active = [True] * len(streams)
    # a buffer containing fetched values in sorted order
    queue = []

    for i, stream in enumerate(streams):
        try:
            heapq.heappush(queue, (next(stream), i))
        except StopIteration:
            active[i] = False

    while queue:
        val, i = heapq.heappop(queue)
        yield val
        if active[i]:
            try:
                heapq.heappush(queue, (next(streams[i]), i))
            except StopIteration:
                active[i] = False
        
    for val, _ in queue:
        yield val


In [54]:
stream1 = range(0, 10, 3)
stream2 = range(1, 10, 3)
stream3 = range(2, 10, 3)

for x in merge_sorted_streams(stream1):
    print(x)

0
3
6
9


In [55]:
for x in merge_sorted_streams([], []):
    print(x)

In [56]:
for x in merge_sorted_streams(stream1, stream2, stream3, [], []):
    print(x)

0
1
2
3
4
5
6
7
8
9


## Tree traversal

Unlike linear data structures (Array, Linked List, Queues, Stacks, etc) which have only one logical way to traverse them, trees can be traversed in different ways. Following are the generally used ways for traversing trees.

```
      1
    /  \ 
   2    3
  / \
 4   5
```

Depth First Traversals: 
  * (a) Inorder (Left, Root, Right) : 4 2 5 1 3
  * (b) Preorder (Root, Left, Right) : 1 2 4 5 3
  * (c) Postorder (Left, Right, Root) : 4 5 2 3 1

Define a Tree class with a method that can walk through the tree in different orders. Hint: use generator will make your life a lot easier.

```python
class TreeNode:
    
    ...

    def in_order(self):
        pass
        
    def pre_order(self):
        pass
        
    def post_order(self):
        pass

    
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)

>>> print(' -> '.join(item for item in root.in_order()))
4 -> 2 -> 5 -> 1 -> 3    


```
   

In [58]:
class TreeNode:

    def __init__(self, val) -> None:
        self.val = val
        self.left = None
        self.right = None

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.val})"
    
    def __str__(self) -> str:
        return str(self.val)
        
    def in_order(self):
        if self.left:
            for val in self.left.in_order():  # or use yield from
                yield val
        yield self.val
        if self.right:
            for val in self.right.in_order():
                yield val
        
    def pre_order(self):
        yield self.val
        if self.left:
            for val in self.left.pre_order():
                yield val
        if self.right:
            for val in self.right.pre_order():
                yield val

    def post_order(self):
        if self.left:
            for val in self.left.post_order():
                yield val
        if self.right:
            for val in self.right.post_order():
                yield val
        yield self.val


In [59]:
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)

root

TreeNode(1)

In [60]:
print(' -> '.join(str(item) for item in root.in_order()))

4 -> 2 -> 5 -> 1 -> 3


In [61]:
print(' -> '.join(str(item) for item in root.pre_order()))

1 -> 2 -> 4 -> 5 -> 3


In [62]:
print(' -> '.join(str(item) for item in root.post_order()))

4 -> 5 -> 2 -> 3 -> 1


## Implement a timer
Implement a timer that can print the execution time of your code. Try to implement it both as a decorator and as a context manager to compare the implementations. Can you implement it using one single class? 

Example:
```python
import time


with timer():
    time.sleep(3)

# Total execution time: 3.000123 seconds (a made up number)


@timer
def sleep(secs):
    time.sleep(secs)
    
sleep(3)
# Total execution time: 3.000123 seconds (a made up number)
```

Below is the code snippet to measure the time:
```python    
import time
start_time = time.time()
main()
print(f"--- {time.time() - start_time} seconds ---"
```

In [63]:
import contextlib
import time


class Timer(contextlib.ContextDecorator):

    def __init__(self):
        self.start_time = None
        
    def __enter__(self):
        if self.start_time:
            raise RuntimeError("Already in Timed context, cannot enter again!")
        self.start_time = time.time()
        return self

    def __exit__(self, *exc):
        print(f"--- Execution time: {time.time() - self.start_time} seconds ---")
        self.start_time = None
    

def timed(func=None):
    
    if func is None:
        print("Using timed as a context manager")
        return Timer()
    
    print("Uing timed as a function decorator")
    return Timer()(func)

In [64]:
with timed():
    time.sleep(1)

Using timed as a context manager
--- Execution time: 1.0049338340759277 seconds ---


In [65]:
@timed
def sleep(secs):
    time.sleep(secs)

sleep(1)

Uing timed as a function decorator
--- Execution time: 1.0029470920562744 seconds ---
